# Introduction

Het gebruik van sociale media is de laatste jaren in enorm tempo toegenomen. Alhoewel het gebruik van sociale media de mogelijkheid biedt voor voorbending en entertainment, echter lijkt het gebruik hiervan ook een keerzijde te hebben. Tal van studies tonen aan dat excessief sociale media gebruik samenhangt met verminderde mentale gezondheid. Zo leggen studies het verband tussen sociale mediaverslaving en mentale problemen, als depressie en angststoornissen (Ergün et al., 2023). Desalniettemin zijn er ook studies die het overtuigt zijn, dat dit verband in werkelijk niet bestaat. Zo concludeert Huang (2018) uit een meta-analyse dat het bestaan van dit verband te snel wordt getrokken en dubieus van aard is. 
Nadat er vanuit artsen, wetenschappers en deskundigen een brandbrief werd verstuurd met een oproep tot regelgeving omtrent smartphone bezit en sociale media gebruik, komt de Nederlandse politiek binnenkort met een richtlijn omtrent dit  (NOS, 2025). In hoeverre is de invoering van deze richtlijnen terecht? Zijn richtlijnen alleen voor kinderen nodig of mooeten deze voor de hele samenleving gelden?

Wereldwijd is de populatie met mentale stoornissen significant gestegen. In de grafiek boven is per land de trend te zien. Vandaar dat dit onderzoek nog meer van belang is. Vrouwen en mannen zijn beide slachtoffer van deze chronische ziekte. 

In dit onderzoek worden beide standpunten onderbouwd aan de hand van data visualisaties, specifiek gericht op sociale mediaverslaving onder studenten. Dit wordt gedaan met behulp van verschillende datasets van diverse groepen studenten. Tevens wordt een dataset van het Institute for Health Metrics and Evaluation gebruikt om de grootschaligheid van het wereldwijde probleem van mentale stoornissen en verminderde mentale gesteldheid in kaart te brengen.

- Douwe Maljers (15264424)
- Justin van Egmond (15011178)
- Noor Verbrugge (15168298)